In [43]:
import requests
import pandas as pd 
import json
from dateutil import parser



In [44]:
API_KEY = "00d239aeae8ecbaf50a4c1352c1837eb-73fbde84df7b82be7a15845f729b3abe"
ACCOUNT_ID = "101-004-30986958-001"
OANDA_URL= "https://api-fxpractice.oanda.com/v3"

In [45]:
session.headers.update({
"Authorization": f"Bearer {API_KEY}",
"Content-Type": "application/json"
})

In [46]:
session = requests.Session()

In [47]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )  
    
    response = session.get(url, params = params, data = None, headers = None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data 


def get_candles_df(data):
    if len(data)== 0:
        return pd.DataFrame. empty 
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data =[]
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['volume'] = candle['volume']
        new_dict['time'] = parser.parse(candle['time'])
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = candle[p][o]
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data =fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed, pair_name, data")
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max}")

In [ ]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)


In [38]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2025-02-07 02:00:00+00:00 <bound method Series.max of 0   2025-02-07 02:00:00+00:00
1   2025-02-07 06:00:00+00:00
2   2025-02-07 10:00:00+00:00
3   2025-02-07 14:00:00+00:00
4   2025-02-07 18:00:00+00:00
5   2025-02-09 22:00:00+00:00
6   2025-02-10 02:00:00+00:00
7   2025-02-10 06:00:00+00:00
8   2025-02-10 10:00:00+00:00
Name: time, dtype: datetime64[ns, tzutc()]>


In [50]:
our_curr = ['EUR', 'USD', 'GBP',  'JPY', 'CHF', 'NZD','CAD', 'AUD' ]

In [ ]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

: 